### Importando bibliotecas

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
import os
from uuid import uuid4
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


### Carrega todos os arquivos PDF do diretório "data" e os divide em documentos individuais

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

data_dir = "data"
loader = PyPDFDirectoryLoader(data_dir)
documents = loader.load_and_split()



### Instanciando a LLM e a classe de Embeddings

In [ ]:
llm = ChatOpenAI(openai_api_key=api_key,model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

### **index = faiss.IndexFlatL2(len(embeddings.embed_query("example query")))**
- `faiss.IndexFlatL2()`: Essa é uma função da biblioteca FAISS (Facebook AI Similarity Search) que cria um índice de vetor plano usando a distância L2 (Euclidiana) como métrica de similaridade.

- `len(embeddings.embed_query("example query"))`: Essa parte do código chama o método `embed_query()` da classe de embeddings (por exemplo, `OpenAIEmbeddings` ou `HuggingFaceEmbeddings`) e retorna o comprimento do vetor de embedding resultante. Isso é usado para definir o número de dimensões do índice FAISS.

Então, essa linha de código está criando um índice FAISS que usa a distância L2 como métrica de similaridade e tem um número de dimensões igual ao comprimento do vetor de embedding retornado pela chamada `embeddings.embed_query("example query")`.

Esse índice FAISS pode ser usado posteriormente para realizar buscas de similaridade em um conjunto de documentos vetorizados. O índice FAISS é otimizado para realizar essas buscas de maneira eficiente, mesmo em conjuntos de dados muito grandes.


In [ ]:
index = faiss.IndexFlatL2(len(embeddings.embed_query("example query")))


### Configuração e Mapeamento de Documentos para Armazenamento e Busca de Similaridade com FAISS
- `index_to_docstore_id = {i: str(uuid4()) for i in range(len(documents))}`:
  - Essa linha cria um dicionário que mapeia o índice de cada documento em `documents` para um UUID (Universally Unique Identifier) único.
  - O `uuid4()` gera um UUID aleatório e único para cada documento.
  - Esse mapeamento será usado posteriormente para conectar os documentos aos seus respectivos vetores de embedding no armazenamento de vetores.

- `docstore = InMemoryDocstore({index_to_docstore_id[i]: doc for i, doc in enumerate(documents)})`:
  - Essa linha cria uma instância da classe `InMemoryDocstore`, que é um armazenamento de documentos em memória.
  - O dicionário criado anteriormente (`index_to_docstore_id`) é usado para mapear cada documento em `documents` para seu respectivo UUID no armazenamento de documentos.

- `vector_store = FAISS(embedding_function=embeddings, index=index, docstore=docstore, index_to_docstore_id=index_to_docstore_id)`:
  - Essa linha cria uma instância da classe `FAISS`, que é um armazenamento de vetores baseado na biblioteca FAISS.
  - `embedding_function=embeddings`: Define a função de embedding a ser usada para gerar os vetores de embedding dos documentos.
  - `index=index`: Define o índice FAISS criado anteriormente.
  - `docstore=docstore`: Define o armazenamento de documentos a ser usado.
  - `index_to_docstore_id=index_to_docstore_id`: Define o mapeamento entre os índices dos documentos e seus respectivos UUIDs no armazenamento de documentos.


In [4]:
index_to_docstore_id = {i: str(uuid4()) for i in range(len(documents))}
docstore = InMemoryDocstore({index_to_docstore_id[i]: doc for i, doc in enumerate(documents)})
vector_store = FAISS(embedding_function=embeddings, index=index, docstore=docstore, index_to_docstore_id=index_to_docstore_id)

### Adiciona os documentos no VDB

In [ ]:
vector_store.add_documents(documents)


In [ ]:
print("There are", len(vector_store.docstore._dict), "docs in the collection")

### Escervendo os *PROMPTS*

In [ ]:
condense_question_system_template = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


### *HistoryAwareRetriever* é um tipo especial de recuperador (retriever) que leva em conta o histórico da conversa

In [ ]:
condense_question_prompt = ChatPromptTemplate.from_messages([
    ("system", condense_question_system_template),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])

history_aware_retriever = create_history_aware_retriever(
    llm, vector_store.as_retriever(), condense_question_prompt
)

- **StuffDocumentsChain**:
  - É um tipo de cadeia (chain) que passa uma lista de documentos para um modelo de linguagem (LLM) usando um prompt específico, como `qa_prompt`.
  - Essa cadeia é projetada para "encher" o prompt com os documentos fornecidos, permitindo que o modelo de linguagem processe e gere respostas com base nas informações desses documentos.

- **RetrievalChain**:
  - É um tipo de cadeia que combina um recuperador (retriever) e uma cadeia de perguntas e respostas (Q&A).
  - O recuperador é responsável por buscar documentos relevantes de uma base de dados ou de um conjunto de documentos.
  - A cadeia de perguntas e respostas, então, utiliza esses documentos recuperados para gerar respostas precisas para as perguntas feitas.


In [ ]:
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])

qa_chain = create_stuff_documents_chain(llm, qa_prompt)

convo_qa_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

In [ ]:
result = convo_qa_chain.invoke({"input": "Me de um resumo do artigo", "chat_history": []})

In [17]:
result["answer"]

'O artigo discute a aplicação da Lei com foco na justiça, reconhecendo que a perfeição da Lei é inatingível, mas defendendo que sua aplicação deve ser justa. Mostra visualizações de atenção em camadas específicas de um modelo, destacando como certas cabeças de atenção lidam com resolução de anáfora e estrutura frasal. Além disso, cita referências de trabalhos anteriores sobre processamento de linguagem natural e aprendizado de máquina.'